paper: https://link.springer.com/article/10.1007/s11111-024-00452-9#Sec10

Prophet: https://facebook.github.io/prophet/docs/quick_start.html#python-api  
github: https://github.com/facebook/prophet  

| **Year**    | **Country/Region** | **Event / Policy**                     | **Migration Impact**                                                 |
| ----------- | ------------------ | -------------------------------------- | -------------------------------------------------------------------- |
| **2019**    | Argentina          | Severe financial crisis; inflation 54% | Argentinians start emigrating; fewer Venezuelans choose Argentina    |
|             | Venezuela          | Hyperinflation + shortages             | Mass exodus (\~4M migrants)                                          |
|             | Chile/Peru/Ecuador | Visa rules tighten                     | Venezuelan migration shifts patterns                                 |
|             | US–Mexico          | MPP (“Remain in Mexico”)               | Slows U.S. asylum, traps many                                        |
| **2020**    | Regional           | COVID closures                         | Migration slows temporarily                                          |
|             | U.S.               | Title 42 begins                        | Migrant expulsions surge                                             |
| **2021 Q1** | Colombia           | TPS regularization for Venezuelans     | Encourages local settlement                                          |
| **2021 Q2** | Chile              | New immigration law (April)            | Triggers Venezuelan & Haitian outflows                               |
| **2021 Q3** | Haiti              | Assassination + earthquake             | Haitian migration peaks                                              |
| **2022 Q1** | Argentina          | Inflation accelerates (\~55%)          | Argentinians emigrate; Venezuelans in Argentina begin leaving        |
| **2022 Q3** | Ecuador            | Venezuelan regularization              | Slows outflow temporarily                                            |
| **2022 Q4** | US–Mexico          | Title 42 expanded to Venezuelans       | Thousands stranded in Mexico                                         |
|             | Argentina          | Inflation \~95%, IMF tensions          | Argentinian emigration accelerates sharply                           |

In [ ]:
import os
import gc
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path

# Geospatial
import geopandas as gpd
import dask_geopandas as dgpd
import rasterio
from rasterstats import zonal_stats
from osgeo import gdal, osr

# Statistics & Bayesian modeling
import pymc as pm
import arviz as az

# Visualisation
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# Base directories
BASE_DIR = Path("/Users/wenlanzhang/PycharmProjects/Mapineq/src/")
DATA_DIR = Path("/Users/wenlanzhang/Downloads/PhD_UCL/Data/Oxford")

country_list = ['CL', 'BR', 'ES', 'PE', 'US', 'EC', 'AR'] # 'CO', 

In [ ]:
df = pd.read_csv(DATA_DIR/f"Migration/international_migration_flow.csv") 
df['year'] = pd.to_datetime(df['migration_month']).dt.year
df['month'] = pd.to_datetime(df['migration_month']).dt.month
df["migration_month"] = pd.to_datetime(df["migration_month"])

# Define the mapping of old codes to new codes
country_code_mapping = {
    'GR': 'EL',  # Greece (GR → EL)
    'GB': 'UK'   # United Kingdom (GB → UK)
}

# Apply the replacement to both columns
df['country_from'] = df['country_from'].replace(country_code_mapping)
df['country_to'] = df['country_to'].replace(country_code_mapping)

df
# len(df['country_from'].unique())

# VE 

## VE 

In [ ]:
df_ve_in = df[(df['country_to'] == 'VE') & (df['num_migrants'] != 0)]
df_ve_out = df[(df['country_from'] == 'VE') & (df['num_migrants'] != 0)]

In [ ]:
# df_ve_in
fig = px.line(
    df_ve_in,
    x="migration_month",
    y="num_migrants",
    color="country_from", 
    markers=True,
    title="Migrants to VE Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

In [ ]:
# df_ve_out
fig = px.line(
    df_ve_out,
    # filtered_df,
    x="migration_month",
    y="num_migrants",
    color="country_to", 
    markers=True,
    title="Migrants from VE Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="To Country"
)

fig.show()

## Haiti

In [ ]:
df_HT_in = df[(df['country_to'] == 'HT') & (df['num_migrants'] != 0)]
df_HT_out = df[(df['country_from'] == 'HT') & (df['num_migrants'] != 0)]

In [ ]:
# df_HT_in
fig = px.line(
    df_HT_in,
    x="migration_month",
    y="num_migrants",
    color="country_from", 
    markers=True,
    title="Migrants to HT Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

In [ ]:
# df_HT_out
fig = px.line(
    df_HT_out,
    x="migration_month",
    y="num_migrants",
    color="country_to", 
    markers=True,
    title="Migrants from HT Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="To Country"
)

fig.show()

## AR

In [ ]:
df_AR_in = df[(df['country_to'] == 'AR') & (df['num_migrants'] != 0)]
df_AR_out = df[(df['country_from'] == 'AR') & (df['num_migrants'] != 0)]

In [ ]:
# df_AR_in
fig = px.line(
    df_AR_in,
    x="migration_month",
    y="num_migrants",
    color="country_from", 
    markers=True,
    title="Migrants to AR Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

In [ ]:
# df_AR_out
fig = px.line(
    df_AR_out,
    x="migration_month",
    y="num_migrants",
    color="country_to", 
    markers=True,
    title="Migrants from AR Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="To Country"
)

fig.show()

# Importing 

## CO

| **Date**         | **Policy Change**                                                                                                   | **Impact on VE Migration**                                                                                                                                                                                                                                             |                                                                                   |
| ---------------- | ------------------------------------------------------------------------------------------------------------------- | ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------- |
| **January 2019** | Colombia imposed entry bans on individuals with close ties to the Maduro regime, including high-ranking officials.  | This measure targeted specific individuals rather than the general Venezuelan population, aiming to restrict the entry of regime-linked figures. It did not directly affect the broader flow of Venezuelan migrants.                                                   |                                                                                   |
| **August 2020**  | Launch of the Temporary Protection Statute (TPS) for Venezuelan migrants.                                           | The TPS provided legal status to undocumented Venezuelan migrants, granting them work permits, access to healthcare, and education. This initiative aimed to regularize the status of Venezuelans residing in Colombia and facilitate their integration into society.  |                                                                                   |
| **October 2021** | Initiation of the Single Registry of Venezuelan Migrants (RUMV).                                                    | The RUMV was established to collect data on Venezuelan migrants in Colombia, aiding in the implementation of the TPS and other support measures. It helped identify the needs and vulnerabilities of the migrant population.                                           |                                                                                   |
| **May 2022**     | Deadline for Venezuelan migrants to register under the TPS.                                                         | Migrants who registered by this date were granted legal status, enabling them to access various services and protections. Those who failed to register faced potential legal challenges and limited access to support.                                                 |                                                                                   |
| **October 2022** | Introduction of the PEP-Tutor program.                                                                              | The PEP-Tutor provided legal status to guardians of Venezuelan children already residing in Colombia, facilitating family reunification and support. It aimed to regularize the status of caregivers and ensure the well-being of minors.                              | ([Wikipedia][1], [UNDP][2], [Wikipedia][3], [The World Bank][4], [as-coa.org][5]) |

[1]: https://en.wikipedia.org/wiki/Sanctions_during_the_Venezuelan_crisis?utm_source=chatgpt.com "Sanctions during the Venezuelan crisis"
[2]: https://www.undp.org/sites/g/files/zskgke326/files/2022-11/PNUDLAC-working-paper-34-Colombia-EN.pdf?utm_source=chatgpt.com "[PDF] Migration in Colombia and Public Policy Responses"
[3]: https://en.wikipedia.org/wiki/Venezuelan_refugee_crisis?utm_source=chatgpt.com "Venezuelan refugee crisis"
[4]: https://thedocs.worldbank.org/en/doc/7277e925bdaa64d6355c42c897721299-0050062023/original/WDR-Colombia-Case-Study-FORMATTED.pdf?utm_source=chatgpt.com "[PDF] Colombia case study of migration from Venezuela - The World Bank"
[5]: https://www.as-coa.org/articles/explainer-venezuelan-migration-policy-americas?utm_source=chatgpt.com "Explainer: Venezuelan Migration Policy in the Americas - AS/COA"


| **Date** | **Policy / Program**                          | **Impact on Venezuelan Migration**                                                  |
| -------- | --------------------------------------------- | ----------------------------------------------------------------------------------- |
| Jan 2019 | Entry restrictions on Maduro officials        | Limited entry for regime-linked individuals; general migrants unaffected            |
| Aug 2020 | Temporary Protection Statute (TPS)            | Legalized status for undocumented Venezuelans; allowed work, healthcare, education  |
| Oct 2021 | Single Registry of Venezuelan Migrants (RUMV) | Collected data to support TPS and aid programs; better migrant tracking             |
| May 2022 | TPS registration deadline                     | Migrants registering gained legal status; others remained irregular                 |
| Oct 2022 | PEP-Tutor program                             | Legal status for guardians of Venezuelan children; facilitated family reunification |


| **Date**             | **Policy / Program**                                    | **Impact on Venezuelan Migration**                                                              |                                                                                                      |
| -------------------- | ------------------------------------------------------- | ----------------------------------------------------------------------------------------------- | ---------------------------------------------------------------------------------------------------- |
| **February 8, 2021** | Announcement of Temporary Protection Statute (ETPV)     | Legalized status for undocumented Venezuelans; allowed work, healthcare, education              |                                                                                                      |
| **March 1, 2021**    | Decree 216: Official implementation of ETPV             | Formalized legal framework for ETPV; began registration process                                 |                                                                                                      |
| **May 2021**         | Start of ETPV registration period                       | Venezuelans began registering for ETPV; access to services initiated                            |                                                                                                      |
| **June 2021**        | ETPV registration deadline                              | Final date for registration; migrants without ETPV status faced legal uncertainty               |                                                                                                      |
| **October 2021**     | Launch of Single Registry of Venezuelan Migrants (RUMV) | Collected data to support ETPV and aid programs; better migrant tracking                        |                                                                                                      |
| **June 18, 2024**    | Announcement of PEP-Tutor program                       | Planned to grant legal status to guardians of Venezuelan minors; aimed to integrate caregivers  |                                                                                                      |
| **October 2024**     | Implementation of PEP-Tutor program                     | Guardians of Venezuelan minors began receiving legal status; facilitated family reunification   | ([Axios][1], [Bitfinance][2], [Migration Network][3], [Cancillería][4], [r4v.info][5], [Reuters][6]) |

[1]: https://www.axios.com/2021/02/08/colombia-venezuelan-migrants-legal-protections?utm_source=chatgpt.com "Colombia to offer temporary protective status to Venezuelan migrants"
[2]: https://bitfinance.news/en/colombia-issues-decree-to-legalize-guardians-of-venezuelan-migrant-children/?utm_source=chatgpt.com "Colombia issues decree to legalize guardians of Venezuelan ..."
[3]: https://migrationnetwork.un.org/pledge/implementation-and-socialization-temporary-protection-statute-venezuelan-migrants-etpv?utm_source=chatgpt.com "Implementation and socialization of the Temporary Protection ..."
[4]: https://www.cancilleria.gov.co/sites/default/files/FOTOS2020/abc_estatuto_al_migrante-ingles-ok.pdf?utm_source=chatgpt.com "[PDF] ABC Temporary Protection Status for Venezuelan Migrants"
[5]: https://www.r4v.info/sites/g/files/tmzbdl2426/files/2021-06/Venezuelans%20in%20Colombia_2021%20%28EN%29_17062021.pdf?utm_source=chatgpt.com "[PDF] Venezuelans in Colombia_2021 (EN)_17062021"
[6]: https://www.reuters.com/world/americas/colombia-give-legal-status-up-540000-venezuelan-migrants-2024-06-18/?utm_source=chatgpt.com "Colombia to give legal status to up to 540,000 Venezuelan migrants"


TPS = ETPV

| **Year**        | **Policy/Event**                                                                                                                             | **Significance**                                                                                                                                               |
| --------------- | -------------------------------------------------------------------------------------------------------------------------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **2017**        | **Introduction of the Permiso Especial de Permanencia (PEP)** via Resolution 1272 (July 28, 2017) following Resolution 5797 (July 25, 2017). | First wide-reaching temporary permit allowing Venezuelans present by that date to legally stay and access basic services. ([Cancillería][1])                   |
| **2017–2020**   | **Multiple extensions of PEP** through subsequent resolutions to accommodate new arrival waves (e.g., 2018, 2019, 2020).                     | Kept the program active and expanded eligibility dates. ([Cancillería][1], [Brigard Urrutia][2])                                                               |
| **Mar 1, 2021** | **Launch of the Estatuto Temporal de Protección para Migrantes Venezolanos (ETPV)** through Decree 216.                                      | Introduced a robust, long-term 10-year legal status, replacing PEP’s temporary framework. ([Forvis Mazars][3], [Pathfinders][4], [migrationnetwork.un.org][5]) |
| **April 2021**  | **Implementation phase kicks off**—Resolution 0971 initiated formal rollout via Migrants Registry starting May 5.                            | Operationalized ETPV logistics to include eligible individuals. ([migrationnetwork.un.org][5])                                                                 |
| **2022**        | **Residency pathway solidified**—Resolution 5477 (2022) established a pathway from PPT/ETPV to Resident Visa.                                | Offered a route toward permanent legal status for holders. ([Forvis Mazars][3])                                                                                |
| **Dec 4, 2024** | **Introduction of Transitional V Visa** (Resolution 12509).                                                                                  | Created a bridge for Venezuelans not previously registered under ETPV to access temporary status, work, and basic services. ([Forvis Mazars][3])               |

[1]: https://www.cancilleria.gov.co/sites/default/files/FOTOS2020/resolucion_cancilleria_nuevo_plazo_de_expedicion_pep_07092020.pdf?utm_source=chatgpt.com "Microsoft Word - Resolución Cancillería nuevo plazo de expedición PEP ..."
[2]: https://www.bu.com.co/en/insights/noticias/special-permit-permanence-pep-modified-terms?utm_source=chatgpt.com "Special Permit of Permanence (PEP): modified terms"
[3]: https://www.forvismazars.com/co/en/who-we-are/news-publications-and-media/our-publications/laboral-y-migratorio/regularization-in-colombia-for-venezuelan-citizen?utm_source=chatgpt.com "Path to migrant regularization in Colombia for Venezuelan citizen ..."
[4]: https://www.sdg16.plus/policies/temporary-protection-status-for-venezuelan-migrants-colombia/?utm_source=chatgpt.com "Temporary Protection Status for Venezuelan Migrants: Colombia - Pathfinders"
[5]: https://migrationnetwork.un.org/pledge/implementation-and-socialization-temporary-protection-statute-venezuelan-migrants-etpv?utm_source=chatgpt.com "Implementation and socialization of the Temporary Protection Statute ..."


people going back: 
1. Policy Stabilization & Registration Cutoff: No new PEPs since March 2021, only renew
2. Economic Challenges in Colombia post covid 
3. Improved (Relative) Conditions in Venezuela since 2022

In [ ]:
df_co_in = df[(df['country_to'] == 'CO') & (df['num_migrants'] != 0)]
df_co_out = df[(df['country_from'] == 'CO') & (df['num_migrants'] != 0)]

In [ ]:
# df_co_in
fig = px.line(
    df_co_in,
    x="migration_month",
    y="num_migrants",
    color="country_from", 
    markers=True,
    title="Migrants to CO Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

In [ ]:
# df_co_out
fig = px.line(
    df_co_out,
    x="migration_month",
    y="num_migrants",
    color="country_to", 
    markers=True,
    title="Migrants from CO Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="To Country"
)

fig.show()

## PE

| **Date**               | **Policy Change**                                                                                                                                                                                        | **Impact on VE Migration**                                                                                                                                                                                                                                                               |                                                                                  |
| ---------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | -------------------------------------------------------------------------------- |
| **June 6, 2019**       | President Martín Vizcarra announced that, starting June 15, 2019, Venezuelans would need a valid passport and visa to enter Peru.                                                                        | This policy shift aimed to regulate the growing influx of Venezuelan migrants. It led to a surge in border crossings as many Venezuelans rushed to enter before the deadline. On June 14, 2019, approximately 9,000 Venezuelans entered Peru through the Tumbes border in a single day.  |                                                                                  |
| **June 15, 2019**      | Implementation of the visa requirement for Venezuelans entering Peru.                                                                                                                                    | Venezuelans without the required documentation were denied entry, leading to increased irregular migration and border challenges. The policy aimed to ensure a more controlled and orderly migration process. However, exceptions were made for vulnerable groups such as children, the elderly, and pregnant women, who were allowed entry without these documents.                                                                            |                                                                                  |
| **July 2021–Nov 2023** | Application period for the Temporary Permanence Permit Card (CPP) for Venezuelan migrants who had arrived in Peru before October 22, 2020, and lacked regular status.                                    | The CPP granted regular status for two years and allowed beneficiaries to transition to one of the regular immigration categories. It aimed to regularize the status of Venezuelan migrants and provide them with legal rights and access to services.                                   |                                                                                  |
| **2022**               | Peru continued efforts to integrate Venezuelan migrants through various programs and policies.                                                                                                           | Ongoing initiatives focused on providing legal status and access to services for Venezuelan migrants, facilitating their integration into Peruvian society.                                                                                                                              | ([Wikipedia][1], [Wikipedia][2], [AS/COA][3], [ECOI][4], [IADB Publications][5]) |

[1]: https://es.wikipedia.org/wiki/Gobierno_de_Mart%C3%ADn_Vizcarra?utm_source=chatgpt.com "Gobierno de Martín Vizcarra"
[2]: https://en.wikipedia.org/wiki/Venezuelan_refugee_crisis?utm_source=chatgpt.com "Venezuelan refugee crisis"
[3]: https://www.as-coa.org/articles/explainer-venezuelan-migration-policy-americas?utm_source=chatgpt.com "Explainer: Venezuelan Migration Policy in the Americas - AS/COA"
[4]: https://www.ecoi.net/en/document/2124532.html?utm_source=chatgpt.com "“Peru and Venezuela: Requirements and procedures for foreign ..."
[5]: https://publications.iadb.org/publications/english/document/Impactregularizationprograminperu.pdf?utm_source=chatgpt.com "[PDF] Impacts of a Regularization Program in Peru - IADB Publications"


1. Temporary Stay Permit (Carnet de Permiso Temporal de Permanencia - CPP) for Venezuelans
Date Implemented: August 2020

2. Reopening of Land Borders
Date Implemented: February 2022

3. Temporary Stay Permit for Haitians
Date Implemented: 2022

| Policy                             | Affected Nationalities | Potential Impact                                                         |                                                                                        |
| ---------------------------------- | ---------------------- | ------------------------------------------------------------------------ | -------------------------------------------------------------------------------------- |
| Temporary Stay Permit (CPP)        | Venezuelans            | Provided legal status and work rights; limited institutional recognition |                                                                                        |
| Reopening of Land Borders          | Venezuelans, Haitians  | Facilitated entry and regularization processes                           |                                                                                        |
| Temporary Stay Permit for Haitians | Haitians               | Offered legal recognition and work rights; challenges in service access  | ([The Global Compact on Refugees \| UNHCR][1], [Migration Network][2], [acnur.org][3]) |

[1]: https://globalcompactrefugees.org/good-practices/ptp-protection-oriented-migratory-arrangement-refugees-peru?utm_source=chatgpt.com "PTP: Protection-oriented migratory arrangement for refugees in Peru"
[2]: https://migrationnetwork.un.org/pledge/implementation-and-socialization-temporary-protection-statute-venezuelan-migrants-etpv?utm_source=chatgpt.com "Implementation and socialization of the Temporary Protection ..."
[3]: https://www.acnur.org/sites/default/files/legacy-pdf/632399d94.pdf?utm_source=chatgpt.com "[PDF] January to June 2022 - ACNUR"


In [ ]:
df_PE_in = df[(df['country_to'] == 'PE') & (df['num_migrants'] != 0)]
df_PE_out = df[(df['country_from'] == 'PE') & (df['num_migrants'] != 0)]

In [ ]:
# df_cl_in
fig = px.line(
    df_PE_in,
    x="migration_month",
    y="num_migrants",
    color="country_from", 
    markers=True,
    title="Migrants to PE Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

In [ ]:
# df_PE_out
fig = px.line(
    df_PE_out,
    x="migration_month",
    y="num_migrants",
    color="country_to", 
    markers=True,
    title="Migrants from PE Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

## CL

| Feature                     | Visa Responsabilidad Democrática (VRD)                 | Consular Tourist Visa                    |
| --------------------------- | ------------------------------------------------------ | ---------------------------------------- |
| **Purpose**                 | Residence and work                                     | Short-term tourism                       |
| **Validity**                | Initially 1 year (renewable)                           | Up to 90 days                            |
| **Renewal / Conversion**    | Renewable; convertible to permanent residency          | Not renewable; limited extension options |
| **Work Authorization**      | Yes                                                    | No                                       |
| **Study Authorization**     | Yes                                                    | Limited/no                               |
| **Access to social rights** | Yes, includes healthcare, education, employment rights | Limited, mostly tourism                  |


| **Date**           | **Policy / Program**                             | **Impact on Venezuelan Migration**                                                         |
| ------------------ | ------------------------------------------------ | ------------------------------------------------------------------------------------------ |
| **June 21, 2019**  | Introduction of Visa Requirement for Venezuelans | Venezuelans now required a visa to enter Chile; irregular crossings increased temporarily. |
| **July 2019**  from 2018？    | Temporary Residency Permit (Visa Temporaria)     | Allowed Venezuelans to regularize their stay, work legally, and access social services.    |
| **June 2021**      | Extension of Temporary Residency Permit          | Continued support for integration; ensured ongoing access to employment and services.      |
| **March 2022**     | New Visa Categories & Regularization Pathways    | Facilitated legal entry and residency; encouraged formal labor market participation.       |


In [ ]:
df_cl_in = df[(df['country_to'] == 'CL') & (df['num_migrants'] != 0)]
df_cl_out = df[(df['country_from'] == 'CL') & (df['num_migrants'] != 0)]

In [ ]:
# df_cl_in
fig = px.line(
    df_cl_in,
    x="migration_month",
    y="num_migrants",
    color="country_from", 
    markers=True,
    title="Migrants to CL Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

In [ ]:
# df_cl_out
fig = px.line(
    df_cl_out,
    x="migration_month",
    y="num_migrants",
    color="country_to", 
    markers=True,
    title="Migrants from CL Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

In [ ]:
df_AR_in = df[(df['country_to'] == 'AR') & (df['num_migrants'] != 0)]
df_AR_out = df[(df['country_from'] == 'AR') & (df['num_migrants'] != 0)]

In [ ]:
# df_cl_in
fig = px.line(
    df_AR_in,
    x="migration_month",
    y="num_migrants",
    color="country_from", 
    markers=True,
    title="Migrants to AR Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

In [ ]:
# df_cl_out
fig = px.line(
    df_AR_out,
    x="migration_month",
    y="num_migrants",
    color="country_to", 
    markers=True,
    title="Migrants from AR Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

## MX

In [ ]:
df_MX_in = df[(df['country_to'] == 'MX') & (df['num_migrants'] != 0)]
df_MX_out = df[(df['country_from'] == 'MX') & (df['num_migrants'] != 0)]

In [ ]:
# df_mx_out
fig = px.line(
    df_MX_out,
    x="migration_month",
    y="num_migrants",
    color="country_to", 
    markers=True,
    title="Migrants from MX Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="To Country"
)

fig.show()

In [ ]:
# df_mx_in
fig = px.line(
    df_MX_in,
    x="migration_month",
    y="num_migrants",
    color="country_from", 
    markers=True,
    title="Migrants to MX Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

## EC

| **Date** | **Policy / Program**                    | **Impact on Venezuelan Migration**                                                           |
| -------- | --------------------------------------- | -------------------------------------------------------------------------------------------- |
| 2019     | Entry requirement: valid passport       | Venezuelans needed passports to enter; some irregular crossings increased                    |
| 2020     | Humanitarian visa for Venezuelans       | Allowed migrants without passports to regularize status; access to work and services         |
| 2021     | Extension of Temporary Residency Permit | Gave Venezuelans legal status and protection; helped integration into Ecuadorian society     |
| 2022     | Simplified work permit process          | Improved access to employment for Venezuelan migrants; encouraged formal labor participation |


| **Date Introduced** | **Policy / Event**                                     | **When in Force**                      | **Target Group / Effect**                                                                                                                 | **Duration / Status**                                                                |
| ------------------- | ------------------------------------------------------ | -------------------------------------- | ----------------------------------------------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------ |
| **25 July 2019**    | **Presidential Decree 826**                            | **From 26 Aug 2019 onwards**           | Introduced **strict entry requirements** for Venezuelans: passport + apostilled criminal record + \$50 fee.                               | **Still in effect** during 2019–2022.                                                |
| **26 Aug 2019**     | **VERHU (Visa de Excepción por Razones Humanitarias)** | **Aug 2019 – 13 Mar 2020** (1st phase) | Allowed Venezuelans who entered **before 26 July 2019** to regularize status.                                                             | Paused in March 2020 due to COVID. Reopened later, extended until **July 27, 2022**. |
| **Oct 2019**        | **Xenophobic backlash & local controls**               | **Oct 2019 onwards**                   | After a Venezuelan migrant committed a femicide, Ecuador imposed **heightened police & ID checks** on Venezuelans.                        | Intensified through **2020–2021**, but no formal law change.                         |
| **16 Mar 2020**     | **COVID-19 Border Closures**                           | **16 Mar 2020 – June 2021**            | Ecuador **closed official borders**, blocking new legal entries. Venezuelans increasingly used irregular crossings.                       | Gradual reopening started mid-2021.                                                  |
| **Mar 2021**        | **End of Moreno presidency**                           | N/A                                    | Transition to Guillermo Lasso in **May 2021**, but policies remained restrictive until new program launched.                              | N/A                                                                                  |
| **1 June 2022**     | **VIRTE (Visa de Residencia Temporal de Excepción)**   | **1 Sep 2022 – 15 Aug 2023**           | Broader regularization program for Venezuelans who entered **before Aug 26, 2019**. Cost **\$50**; required biometric registration first. | Active **Sep 2022 to Aug 2023**, granting over **62,000 visas**.                     |


In [ ]:
df_EC_in = df[(df['country_to'] == 'EC') & (df['num_migrants'] != 0)]
df_EC_out = df[(df['country_from'] == 'EC') & (df['num_migrants'] != 0)]

In [ ]:
# df_EC_in
fig = px.line(
    df_EC_in,
    x="migration_month",
    y="num_migrants",
    color="country_from", 
    markers=True,
    title="Migrants to EC Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

In [ ]:
# df_EC_out
fig = px.line(
    df_EC_out,
    x="migration_month",
    y="num_migrants",
    color="country_to", 
    markers=True,
    title="Migrants from EC Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="To Country"
)

fig.show()

## UY (with AR) 

In [ ]:
df_UY_in = df[(df['country_to'] == 'UY') & (df['num_migrants'] != 0)]
df_UY_out = df[(df['country_from'] == 'UY') & (df['num_migrants'] != 0)]

In [ ]:
# df_ur_out
fig = px.line(
    df_UY_out,
    x="migration_month",
    y="num_migrants",
    color="country_to", 
    markers=True,
    title="Migrants from UY Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

In [ ]:
# df_uy_in
fig = px.line(
    df_UY_in,
    x="migration_month",
    y="num_migrants",
    color="country_from", 
    markers=True,
    title="Migrants to UY Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

## DO (HT) 

In [ ]:
df_DO_in = df[(df['country_to'] == 'DO') & (df['num_migrants'] != 0)]
df_DO_out = df[(df['country_from'] == 'DO') & (df['num_migrants'] != 0)]

In [ ]:
# df_DO_in
fig = px.line(
    df_DO_in,
    x="migration_month",
    y="num_migrants",
    color="country_from", 
    markers=True,
    title="Migrants to DO Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

In [ ]:
# df_DO_out
fig = px.line(
    df_DO_out,
    x="migration_month",
    y="num_migrants",
    color="country_to", 
    markers=True,
    title="Migrants from DO Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

# MM to TH

In [ ]:
df_MM_in = df[(df['country_to'] == 'MM') & (df['num_migrants'] != 0)]
df_MM_out = df[(df['country_from'] == 'MM') & (df['num_migrants'] != 0)]

fig = px.line(
    df_MM_in,
    x="migration_month",
    y="num_migrants",
    color="country_from", 
    markers=True,
    title="Migrants to MM Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

In [ ]:
# df_MM_out
fig = px.line(
    df_MM_out,
    x="migration_month",
    y="num_migrants",
    color="country_to", 
    markers=True,
    title="Migrants from MM Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

In [ ]:
df_TH_in = df[(df['country_to'] == 'TH') & (df['num_migrants'] != 0)]
df_TH_out = df[(df['country_from'] == 'TH') & (df['num_migrants'] != 0)]

fig = px.line(
    df_TH_in,
    x="migration_month",
    y="num_migrants",
    color="country_from", 
    markers=True,
    title="Migrants to TH Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

In [ ]:
# df_TH_out
fig = px.line(
    df_TH_out,
    x="migration_month",
    y="num_migrants",
    color="country_to", 
    markers=True,
    title="Migrants from TH Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="To Country"
)

fig.show()

# SY TR

In [ ]:
df_SY_in = df[(df['country_to'] == 'SY') & (df['num_migrants'] != 0)]
df_SY_out = df[(df['country_from'] == 'SY') & (df['num_migrants'] != 0)]

# df_DO_in
fig = px.line(
    df_SY_in,
    x="migration_month",
    y="num_migrants",
    color="country_from", 
    markers=True,
    title="Migrants to SY Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

In [ ]:
# df_SY_out
fig = px.line(
    df_SY_out,
    x="migration_month",
    y="num_migrants",
    color="country_to", 
    markers=True,
    title="Migrants from SY Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="To Country"
)

fig.show()

In [ ]:
df_TR_in = df[(df['country_to'] == 'TR') & (df['num_migrants'] != 0)]
df_TR_out = df[(df['country_from'] == 'TR') & (df['num_migrants'] != 0)]

# df_TR_in
fig = px.line(
    df_TR_in,
    x="migration_month",
    y="num_migrants",
    color="country_from", 
    markers=True,
    title="Migrants to TR Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="From Country"
)

fig.show()

In [ ]:
# df_TR_out
fig = px.line(
    df_TR_out,
    x="migration_month",
    y="num_migrants",
    color="country_to", 
    markers=True,
    title="Migrants from TR Over Time"
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Migrants",
    legend_title="To Country"
)

fig.show()